# SPARCL + Jdaviz

This notebook is based on [an example](https://github.com/camipacifici/jdaviz/blob/concept-sparcl/notebooks/concepts/specviz_sparcl.ipynb) originally developed by [Camilla Pacifici](https://github.com/camipacifici).

Packages:

* [SPARCL](https://astrosparcl.datalab.noirlab.edu)
* [Jdaviz](https://jdaviz.readthedocs.io/en/latest/index.html)

Installation tips:

* `pip install sparclclient`
* `pip install jdaviz`

In [ ]:
# SPARCL import
from sparcl.client import SparclClient
# Jdaviz import
from jdaviz import Specviz
# Jupyter
import ipywidgets as widgets
from IPython.display import display
# Specutils import
from specutils import Spectrum1D, SpectrumList
# Astropy import
import astropy.units as u
from astropy.nddata import InverseVariance, StdDevUncertainty

## Set up the SPARCL client

In [ ]:
client = SparclClient()
client.all_datasets

In [ ]:
# client.get_all_fields(dataset_list=['SDSS-DR16'])

In [ ]:
# client.get_all_fields(dataset_list=['DESI-EDR'])

## Run a search

In [ ]:
out = ['sparcl_id', 'ra', 'dec', 'spectype', 'subtype', 'specid', 'redshift', 'redshift_err',]
cons = {'redshift': [2.0, 5.0],
        'redshift_warning': [0],
        'specprimary': [1],
        'spectype': ['QSO'],
        'data_release': ['DESI-EDR'],
        }

In [ ]:
found = client.find(outfields=out,
                    constraints=cons,
                    limit=50)

In [ ]:
for record in found.records:
    if record['_dr'] == 'DESI-EDR':
        record['DESIID'] = f"DESI J{record['ra']:08.4f}{record['dec']:+08.4f}"
metadata = sorted(found.records, key=lambda x: x['DESIID'])
ids = [m['sparcl_id'] for m in metadata]

## Retrieve the records

In [ ]:
specunit = u.Unit('10-17 erg cm-2 s-1 AA-1')
spectra = dict()
entries = [(m['DESIID'], i) for i, m in enumerate(metadata)]
select = widgets.Dropdown(options=entries, value=0, description='Spectrum:',)
output = widgets.Output()
specviz = Specviz()

def on_selected(b):
    key = entries[b['new']][0]
    result = client.retrieve(uuid_list=[ids[b['new']]],
                             include=['specid',
                                      'survey',
                                      'ra',
                                      'dec',
                                      'redshift',
                                      'redshift_err',
                                      'flux', 
                                      'ivar', 
                                      'wavelength', 
                                      'mask', 
                                      'model'])
    if key not in spectra:
        spectra[key] = (Spectrum1D(spectral_axis=result.records[0]['wavelength']*u.AA,
                                   flux=result.records[0]['flux']*specunit,
                                   uncertainty=InverseVariance(result.records[0]['ivar']*(specunit**-2)).represent_as(StdDevUncertainty),
                                   mask=result.records[0]['mask'],
                                   redshift=result.records[0]['redshift']),
                        Spectrum1D(spectral_axis=result.records[0]['wavelength']*u.AA,
                                   flux=result.records[0]['model']*specunit))
    if 'old' in b:
        data0 = specviz.app.data_collection[0]
        data1 = specviz.app.data_collection[1]
        specviz.app.data_collection.remove(data0)
        specviz.app.data_collection.remove(data1)
    specviz.load_data(spectra[key][0], data_label=key + ' Data')
    opt = specviz.plugins['Plot Options']
    opt.layer = key + ' Data'
    opt.line_color.value = '#000000'
    opt.line_as_steps = True
    opt.uncertainty_visible = True
    meta0 = specviz.app.data_collection[0].meta
    meta0['RA'] = result.records[0]['ra']
    meta0['Dec'] = result.records[0]['dec']
    meta0['redshift'] = result.records[0]['redshift']
    meta0['redshift uncertainty'] = result.records[0]['redshift_err']
    # meta0.append(('RA', result.records[0]['ra'], '[J2000.0 deg]'))
    # meta0.metadata.append(('Dec', result.records[0]['dec'], '[J2000.0 deg]'))
    # meta0.metadata.append(('redshift', result.records[0]['redshift'], 'baz'))
    # meta0.metadata.append(('redshift uncertainty', result.records[0]['redshift_err'], 'foo'))
    specviz.load_data(spectra[key][1], data_label=key + ' Model')
    opt = specviz.plugins['Plot Options']
    opt.layer = key + ' Model'
    opt.line_color.value = '#FF0000'
    opt.line_as_steps = True
    m = specviz.plugins['Metadata']
    m.dataset = key + ' Model'
    m.dataset = key + ' Data'
    # with output:
    #     print(spectra)

on_selected({'new': 0})
select.observe(on_selected, names='value')
specviz.show('sidecar:split-right')
display(select, output)

# Notes
We will probably want calls to the API to do some basic stuff like:
- change plot options
- smooth the spectrum
- plot some lines
- load the redshift
- do some line analysis too maybe?